<a href="https://colab.research.google.com/github/ysuter/bme-labs/blob/main/session02/Ultrasound_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification from Ultradound images

This example shows you a simple classification from ultrasouind data. It takes images from the BUSI dataset, where we have information for all images, if they contain a malignant or bening lesion, or if the tissue is healthy. We will try to directly infer this from the images.

To make the data easier to handle, the data is resized to 200x200 pixels. 

## Enabling GPU Support

To use GPU resources through Colab, change the runtime to GPU:

    From the "Runtime" menu select "Change Runtime Type"
    Choose "GPU" from the drop-down menu
    Click "SAVE"

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not). Running

!nvidia-smi

in a cell will verify this has worked and show you what kind of hardware you have access to.


First, we download the data and install some required packages. We will use MONAI and PyTorch.


In [ ]:
!wget https://www.dropbox.com/s/7o43bjdzfuul8l9/dataset-bmelab23.zip

In [ ]:
!unzip dataset-bmelab23.zip
dataroot = "/content/dataset-bmelab23"

In [ ]:
!wget https://www.dropbox.com/s/o4lpi26yufaaljd/busi_datainfo_fs23.csv

In [ ]:
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==0.6.0"
!pip install -qU "pandas"
!pip install -qU "scikit-learn"
!pip install -qU "captum"

Let's check what kind of GPU we have available

In [ ]:
!nvidia-smi

## Data handling
Next, we prepare the data for training 

In [ ]:
import os
import pandas as pd
import numpy as np
from monai.data import ImageDataset

subjectlist = sorted([e for e in os.listdir(dataroot) if 'mask' not in e])
print(subjectlist[:10])

imglist = np.array([os.path.join(dataroot, e) for e in subjectlist])
print(imglist[:10])

imglist_masks = np.array([e.split('.png')[0] + '_mask.png' for e in imglist])

diagnosis = pd.read_csv("/content/busi_datainfo_fs23.csv")

# get encoded diagnosis for each subject
diagnosis_list = np.array([diagnosis.loc[diagnosis["Filename"] == e, :]["Diagnosis_encoded"].values[0] for e in subjectlist])
diagnosis_raw_list = [diagnosis.loc[diagnosis["Filename"] == e, :]["Diagnosis"].values[0] for e in subjectlist]

# get overview of the class distribution
print(diagnosis["Diagnosis"].value_counts())


## Data loading and augmentations
The very high number of parameters in common neural networks can lead to overfitting. To make this harder, we perturb the training data. This is called "data augmentation". In this example, we randomly flip the image, rotate it, and add noise.

In [ ]:
from monai.transforms import Compose, AddChannel, NormalizeIntensity, \
  RandAdjustContrast, RandFlip, RandRotate, RandGaussianNoise

# Data augmentation
train_tfm = Compose([AddChannel(), NormalizeIntensity(),
                     RandAdjustContrast(prob=0.1), 
                     RandFlip(prob=0.3), 
                     RandRotate(prob=0.1, range_x=0.2), # rotation +/- 11.5°
                     RandGaussianNoise(prob=0.1)])

test_tfm = Compose([AddChannel(), NormalizeIntensity()])

Next, we create the dataset and random train/test split. Please note, that we would create train/validation/test splits for a real application, but omit this for the [sake of simplicity](https://xkcd.com/2587/). We reserve 25% of the data for testing.

In [ ]:
from scipy.spatial import transform
from sklearn.model_selection import train_test_split
import numpy as np
from monai.data import ImageDataset, DataLoader

# Split the data into training and testing sets
trainidx, testidx, y_train, y_test = train_test_split(np.arange(len(subjectlist)), 
                                                      diagnosis_list,
                                                      test_size=0.25, 
                                                      shuffle=True,
                                                      stratify=diagnosis_list,
                                                      random_state=42)

print("Size of the training and test datasets")
print(len(trainidx))
print(len(testidx))

batchsize = 200

# create a validation data set and loader
trainingset = ImageDataset(
    image_files=imglist[trainidx],
    labels=diagnosis_list[trainidx],
    seg_files=imglist_masks[trainidx],
    transform=train_tfm,
    image_only=False,
    transform_with_metadata=False,
)
train_loader = DataLoader(trainingset, batch_size=batchsize, shuffle=True, num_workers=2, pin_memory=True)

# create a validation data set and loader
testset = ImageDataset(
    image_files=imglist[testidx],
    labels=diagnosis_list[testidx],
    seg_files=imglist_masks[testidx],
    transform=test_tfm,
    image_only=False,
    transform_with_metadata=False,
)
val_loader = DataLoader(testset, batch_size=batchsize, num_workers=2, pin_memory=True)

Make the whole thing reproducible by fixing the seeds

In [ ]:
import torch
import random
torch.backends.cudnn.benchmark = False
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

## The neural network

Current state of the art networks probably have too many learnable parameters for this rather simple task. So we define a simpler network for this classification. It consists of 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.InstanceNorm2d(out_channels),
            nn.PReLU(),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.InstanceNorm2d(out_channels),
            nn.PReLU(),
        )

        self._init_weights()

    def _init_weights(self):
      for m in self.modules():
        if isinstance(m, nn.Conv2d):
          nn.init.kaiming_normal_(m.weight)
          if m.bias is not None:
            nn.init.ones_(m.bias)
          elif isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight)
            if m.bias is not None:
              nn.init.ones_(m.bias)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = F.avg_pool2d(x, 2)
        return x


class Classifier(nn.Module):
    def __init__(self, n_classes, n_input_channels):
        super().__init__()

        self.conv0 = nn.Sequential(
            nn.Conv2d(in_channels=n_input_channels, out_channels=32, kernel_size=7, padding=3, stride=2),
            nn.InstanceNorm2d(32),
            nn.PReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        self.featextractor = nn.Sequential(
            ConvBlock(in_channels=32, out_channels=64),
            ConvBlock(in_channels=64, out_channels=128),
            ConvBlock(in_channels=128, out_channels=256),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.drop_layer = nn.Dropout(p=0.2)

        self.fc = nn.Linear(256, n_classes)


    def forward(self, x):
        x = self.conv0(x)
        x = self.featextractor(x)
        x = x.view(x.size(0), -1)
        x = self.drop_layer(x)
        out = self.fc(x)

        return out

Let's look at the classification model we created:

In [ ]:
from torchsummary import summary
import torch 

# use a GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Classifier(n_classes=3, n_input_channels=1)
model.to(device)
print(model)

summary(model, input_size=(1,200,200), batch_size=batchsize)

## The loss and optimizer

Next, we need a loss fuction and an optimizer. We will use the cross-entropy loss and the "Adam" optimizer.

In [ ]:
learning_rate = 4e-3

classificationloss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

## Training
We have everything set up, from the data to the network, so we can finally train our model.

In [ ]:
from monai.metrics import ROCAUCMetric, ConfusionMatrixMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, \
    roc_auc_score, classification_report, ConfusionMatrixDisplay

epoch_num = 60
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = list()
metric_values = list()
auc_metric = ROCAUCMetric()
accuracy = list()
train_loss = list()
val_loss = list()
train_accuracy = list()
val_accuracy = list()

for epoch in range(epoch_num):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{epoch_num}")

    epoch_loss = 0
    epoch_loss_val = 0
    step = 1

    steps_per_epoch = len(trainingset) // train_loader.batch_size
    pred_values = list()
    label_values = list()
    pred_values_test = list()
    label_values_test = list()

    # put the network in train mode; this tells the network and its modules to
    # enable training elements such as normalisation and dropout, where applicable
    model.train()
    for train_batch in train_loader:

      inputs, labels, info = train_batch[0].to(device), \
                              train_batch[2].to(device), \
                              train_batch[3]

      # prepare the gradients for this step's back propagation
      optimizer.zero_grad()
        
      # run the network forwards
      outputs = model(inputs)
        
      # run the loss function on the outputs
      loss = classificationloss(outputs, labels)
        
      # compute the gradients
      loss.backward()
        
      # tell the optimizer to update the weights according to the gradients
      # and its internal optimisation strategy
      optimizer.step()

      label_values += labels.cpu()
      pred_values += outputs.argmax(dim=1).cpu()

      epoch_loss += loss.item()
      # print(f"{step}/{len(trainingset) // trainloader.batch_size + 1}, training_loss: {loss.item():.4f}")
      step += 1

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    train_accuracy.append(accuracy_score(label_values, pred_values))
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
    print("Accuracy: " + str(accuracy_score(label_values, pred_values)))
    
    # switch off training features of the network for this pass
    model.eval()

    # 'with torch.no_grad()' switches off gradient calculation for the scope of its context
    with torch.no_grad():
        # create lists to which we will concatenate the the validation results


        val_step = 0
        # iterate over each batch of images and run them through the network in evaluation mode
        for val_batch in val_loader:
          val_images, val_masks, val_labels, val_info = \
                                    val_batch[0].to(device), \
                                    val_batch[1].to(device), \
                                    val_batch[2].to(device), \
                                    val_batch[3]

          # run the network
          val_out = model(val_images)

          test_loss = classificationloss(val_out, val_labels)
          epoch_loss_val += test_loss.item()

          predicted = val_out.argmax(dim=1).cpu()
          pred_values_test += predicted
          label_values_test += val_labels.cpu()
          val_step += 1

        val_accuracy.append(accuracy_score(label_values_test, pred_values_test))
        epoch_loss_val /= val_step
        val_loss.append(epoch_loss_val)           

print("Done :-)")
       

Now we can check how the loss and the accuracy evolves over the training process:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

epochs = np.arange(1, len(epoch_loss_values) +1)

plt.figure("train/test", (12, 6))

plt.subplot(2, 2, 1)
plt.title("Training loss")
plt.xlabel("epoch")
plt.plot(epochs, epoch_loss_values)

plt.subplot(2, 2, 2)
plt.title("Test loss")
plt.xlabel("epoch")
plt.plot(epochs, val_loss)

plt.subplot(2, 2, 3)
plt.title("Training accuracy")
y = train_accuracy
plt.xlabel("epoch")
plt.plot(epochs, train_accuracy)

plt.subplot(2, 2, 4)
plt.title("Test accuracy")
plt.xlabel("epoch")
plt.plot(epochs, val_accuracy)
plt.suptitle("Ultrasound image classification")
plt.show()

# Evaluate with a classification report and confusion matrix
... like you already did in the first session

In [ ]:
decoding = {0: 'normal', 1: 'benign', 2: 'malignant'}

In [ ]:
print(classification_report(label_values_test, pred_values_test, target_names=decoding.values()))

ConfusionMatrixDisplay.from_predictions(label_values_test, pred_values_test, sample_weight=None,
                                        display_labels=decoding.values(), 
                                        include_values=True, 
                                        xticks_rotation='horizontal', 
                                        values_format=None, cmap='viridis', 
                                        colorbar=True)
plt.title("Confusion matrix")
plt.show()

# And another normalized version
ConfusionMatrixDisplay.from_predictions(label_values_test, pred_values_test, sample_weight=None, 
                                        normalize='all', 
                                        display_labels=decoding.values(), 
                                        include_values=True, 
                                        xticks_rotation='horizontal', 
                                        values_format=None, cmap='viridis', 
                                        ax=None, colorbar=True)
plt.title("Normalized confusion matrix")
plt.show()

# Model interpretation

Now we can investigate what lead the model to the classification decision. We will use the library "Captum" for this.

In [ ]:
from captum.attr import LayerGradCam, LayerAttribution

Now we can calculate the grad cam output. Here that is done for the second layer of the feature extractor. Check what the size of the attribution map is.

In [ ]:
cam = LayerGradCam(model, layer=model.featextractor[1])
attr = cam.attribute(val_images, val_labels)
print(val_images[0].shape)
print(attr[0].shape)

Now we upsample the images to match the input size (200x200 pixels). This can be done for the whole batch (in case this is our whole validation set) with one function call.

In [ ]:
upsampled_attr_val = LayerAttribution.interpolate(attr, (200, 200), "area")

Overlay the grad cam result for the first sample in the validation set.

In [ ]:
# show axial slice
print(val_labels[0])
print(pred_values_test)
print(val_batch[3]['filename_or_obj'][0])
plt.imshow(torch.squeeze(val_images[0][0, :, :]).cpu().detach().numpy(), cmap='gray')
plt.imshow(torch.squeeze(upsampled_attr_val[0]).cpu().detach().numpy(), alpha=0.5)
plt.axis('off')
plt.show()

Let's map the encoded predictions and labes back to text

Now we can visualize the prediction and grad cam output for any sample you like in our validation set. Adapt valindex to select a different one.

In [ ]:
import matplotlib.pyplot as plt

# select another sample with this index (0-194)
valindex = 30

samplename = os.path.split(val_info['filename_or_obj'][valindex])[-1].split('.png')[0]
predicted_label = decoding[int(val_labels[valindex].cpu().numpy())]
gt_label = decoding[int(val_out.argmax(dim=1)[valindex].cpu().numpy())]
print(samplename)

print("Predicted: " + predicted_label)
print("Ground truth: " + gt_label)

plt.figure("Model interpretation", (16, 8))
plt.subplot(1, 2, 1)
plt.title("Interpretation")
plt.imshow(torch.squeeze(val_images[valindex][0, :, :]).cpu().detach().numpy(), cmap='gray')
plt.imshow(torch.squeeze(upsampled_attr_val[valindex]).cpu().detach().numpy(), alpha=0.5)
plt.axis('off')


plt.subplot(1, 2, 2)
plt.title("Label map")
plt.imshow(torch.squeeze(val_masks[valindex]).cpu().detach().numpy(), cmap='gray')
plt.axis('off')

plt.suptitle("Sample " + samplename + " predicted: " + predicted_label + ", true: " + gt_label)
plt.tight_layout()
plt.show()
